# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/osamaalsubaie/Desktop/Data Engineering with AWS/Projects/Data Modeling with Apache Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list 
for f in file_path_list:

# reading csv file
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        next(csvreader)

 # extracting each data row one by one and append it
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

# uncomment the code below if you would like to get total number of rows
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [24]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
                   WITH REPLICATION = 
                   { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query 1:
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

## Data model
Since we need the results based on sessionId and itemInSession, the Primary Key for our table will be the sessionId and the itemInSession.

### Query
The query for designing the data model will be:

```
CREATE TABLE IF NOT EXISTS song_info_by_session
(
        artist text,
        song text,
        length double,
        sessionId int,
        itemlnSession int,
        PRIMARY KEY (sessionId, itemlnSession)
)
```

In [26]:
## since we need to filter by sessionId and itemInSession, we need to make them as the composite primary key
query1 = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query1 = query1 + "(sessionId int, itemlnSession int, artist text, song text, length double, PRIMARY KEY (sessionId, itemlnSession))"

try:
    session.execute(query1)
except Exception as e:
    print(e)

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header

    for line in csvreader:
        # Insert data into song_info_by_session table
        query = "INSERT INTO song_info_by_session (sessionId, itemlnSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        # Assign which column element should be assigned for each column in the INSERT statement
        try:
            session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))
        except Exception as e:
            print(e)

#### SELECT to verify that the data have been inserted into each table

In [28]:
# Select statement to display the results of the song_info_by_session table and answer the question for query 1
query = "SELECT artist, song, length FROM song_info_by_session WHERE sessionId = 338 AND itemlnSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Dataframe to store the results of the query and print it out
data = {'artist':[], 'song':[], 'length':[]}
df = pd.DataFrame(data)

for row in rows:
    df = df.append({'artist':row.artist, 'song':row.song, 'length':row.length}, ignore_index=True)

display(df)

/var/folders/p4/zzt9glrd7n702xpzdmwyb59r0000gn/T/ipykernel_52356/2078190258.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'artist':row.artist, 'song':row.song, 'length':row.length}, ignore_index=True)


,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


# Query 2:
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

## Data model
Since we need the results based on sessionId and sessionId, and Since the data needs to be sorted by itemInSession, the itemInSession should be the clustering key and the Primary Key for our table will be the sessionId and the itemInSession.

### Query
The query for designing the data model will be:

```
CREATE TABLE IF NOT EXISTS song_info_by_user_and_session
(
        userId int,
        sessionId int,
        itemlnSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY ((userId, sessionId), itemlnSession)
)
```

In [29]:
## since we need to filter by userId and sessionId, we need to make them as the partition key
query2 = "CREATE TABLE IF NOT EXISTS song_info_by_user_and_session "
query2 = query2 + """(userId int, sessionId int, itemlnSession int,
                      artist text, song text,
                      firstName text, lastName text,
                      PRIMARY KEY ((userId, sessionId), itemlnSession))"""

try:
    session.execute(query2)
except Exception as e:
    print(e)


In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header

    for line in csvreader:
        # Insert data into song_info_by_user_and_session table
        query = "INSERT INTO song_info_by_user_and_session (userId, sessionId, itemlnSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        # Assign which column element should be assigned for each column in the INSERT statement
        try:
            session.execute(query, (int(line[10]),int(line[8]), int(line[3]),
                                    str(line[0]), str(line[9]),
                                    str(line[1]), str(line[4])))
        except Exception as e:
            print(e)

#### SELECT to verify that the data have been inserted into each table

In [31]:
# Select statement to display the results of the song_info_by_user_and_session table and answer the question for query 2
query = """SELECT artist, song, firstName, lastName FROM song_info_by_user_and_session
           WHERE userId = 10 AND sessionId = 182
           ORDER BY itemlnSession"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)


## Dataframe to store the results of the query and print it out
data = {'artist':[], 'song':[], 'firstname':[], 'lastname':[]}
df = pd.DataFrame(data)

for row in rows:
    df = df.append({'artist':row.artist, 'song':row.song, 'firstname':row.firstname, 'lastname':row.lastname}, ignore_index=True)

display(df)

/var/folders/p4/zzt9glrd7n702xpzdmwyb59r0000gn/T/ipykernel_52356/296415563.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'artist':row.artist, 'song':row.song, 'firstname':row.firstname, 'lastname':row.lastname}, ignore_index=True)
/var/folders/p4/zzt9glrd7n702xpzdmwyb59r0000gn/T/ipykernel_52356/296415563.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'artist':row.artist, 'song':row.song, 'firstname':row.firstname, 'lastname':row.lastname}, ignore_index=True)
/var/folders/p4/zzt9glrd7n702xpzdmwyb59r0000gn/T/ipykernel_52356/296415563.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'artist':row.artist, 'song':row.song, 'firstname':row.firstname, 'lastname':row.l

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


# Query 3:
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Data model
Since we need the results based on song name, firstName and lastName might be not unique, so the song name and the userid will be perfect Primary Key.

### Query
The query for designing the data model will be:

```
CREATE TABLE IF NOT EXISTS song_listeners_info
    (
        firstName text,
        lastName text,
        userId int,
        song text,
        PRIMARY KEY ((userId,song))
     )
```

In [32]:
# since we need to filter by song name, we need to make it as the primary key. However, since the firstName and lastName might not be unique, we need to add the userId to make the primary key unique.
query3 = "CREATE TABLE IF NOT EXISTS song_listeners_info "
query3 = query3 + """(song text, userId int,
                      firstName text, lastName text,
                      PRIMARY KEY (song, userId))"""

try:
    session.execute(query3)
except Exception as e:
    print(e)

                    

In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header

    for line in csvreader:
        # Insert data into song_listeners_info table
        query = "INSERT INTO song_listeners_info (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"

        # Assign which column element should be assigned for each column in the INSERT statement
        try:
            session.execute(query, (str(line[9]), int(line[10]),
                                    str(line[1]), str(line[4])))
                                    
        except Exception as e:
            print(e)

#### SELECT to verify that the data have been inserted into each table

In [34]:
# Select statement to display the results of the song_listeners_info table and answer the question for query 3
query = """SELECT userId, firstName, lastName FROM song_listeners_info
           WHERE song = 'All Hands Against His Own' """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Dataframe to store the results of the query and print it out
data = {'userid':[], 'firstname':[], 'lastname':[]}
df = pd.DataFrame(data)
for row in rows:
    df = df.append({'userid':row.userid, 'firstname':row.firstname, 'lastname':row.lastname}, ignore_index=True)

display(df)

/var/folders/p4/zzt9glrd7n702xpzdmwyb59r0000gn/T/ipykernel_52356/322075261.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'userid':row.userid, 'firstname':row.firstname, 'lastname':row.lastname}, ignore_index=True)
/var/folders/p4/zzt9glrd7n702xpzdmwyb59r0000gn/T/ipykernel_52356/322075261.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'userid':row.userid, 'firstname':row.firstname, 'lastname':row.lastname}, ignore_index=True)
/var/folders/p4/zzt9glrd7n702xpzdmwyb59r0000gn/T/ipykernel_52356/322075261.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'userid':row.userid, 'firstname':row.firstname, 'lastname':row.lastname}, ignore_index=True)


,userid,firstname,lastname
0,29.0,Jacqueline,Lynch
1,80.0,Tegan,Levine
2,95.0,Sara,Johnson


### Drop the tables before closing out the sessions

In [35]:
# Drop the table before closing out the sessions
tables = ['song_info_by_session', 'song_info_by_user_and_session', 'song_listeners_info']

try:
    for table in tables:
        rows = session.execute("drop table {}".format(table))
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()